In [5]:
from statsmodels.tsa.stattools import adfuller, grangercausalitytests, add_constant, coint, pacf
from statsmodels.tsa.vector_ar.vecm import coint_johansen, select_coint_rank
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.api import AutoReg, VAR, VARMAX
from geopy.distance import great_circle
import matplotlib.pyplot as plt
import sklearn.metrics as skm
from hampel import hampel
import geopandas as gpd
import pandas as pd
import numpy as np
import math

In [6]:
data = pd.read_csv(r'/Users/main/Vault/Thesis/Data/pm25_weer.csv')
data["FH"] = data["FH"] * 0.36
data.drop(data.iloc[:, 0:7], axis=1, inplace=True)
data.drop(data.iloc[:, 1:5], axis=1, inplace=True)
{
data.drop(["jaar", "maand", "weeknummer", "#STN", "YYYYMMDD",
"components", "dag", "tijd", "uur", "datum", "sensortype", "weekdag", "U", "H", "T"], 
axis=1, inplace=True)
}

{None}

In [7]:
grouped_df = data.groupby(by=["timestamp", "tag"]).median().copy().reset_index()
grouped_df.rename(columns={"DD":"Angle", "FH":"Wind"}, inplace=True)
grouped_df["Date"] = pd.to_datetime(grouped_df["timestamp"].astype(str))
grouped_df.set_index("Date", inplace=True)
grouped_df

,timestamp,tag,pm25,latitude,longitude,Angle,Wind
Date,,,,,,,
2019-06-25 01:00:00+00:00,2019-06-25T01:00:00.000000Z,Amsterdam,13.725,52.359714,4.866208,50.0,7.2
2019-06-25 02:00:00+00:00,2019-06-25T02:00:00.000000Z,Amsterdam,15.153,52.359714,4.866208,50.0,7.2
2019-06-25 03:00:00+00:00,2019-06-25T03:00:00.000000Z,Amsterdam,12.785,52.359714,4.866208,50.0,7.2
2019-06-25 04:00:00+00:00,2019-06-25T04:00:00.000000Z,Amsterdam,13.421,52.359714,4.866208,70.0,7.2
2019-06-25 05:00:00+00:00,2019-06-25T05:00:00.000000Z,Amsterdam,12.670,52.359714,4.866208,80.0,7.2
...,...,...,...,...,...,...,...
2022-10-22 23:00:00+00:00,2022-10-22T23:00:00.000000Z,IJmuiden,13.206,52.455538,4.600440,140.0,7.2
2022-10-22 23:00:00+00:00,2022-10-22T23:00:00.000000Z,Uithoorn,18.700,52.238091,4.808229,140.0,7.2
2022-10-22 23:00:00+00:00,2022-10-22T23:00:00.000000Z,Velsen-Noord,13.000,52.472797,4.648964,140.0,7.2


In [ ]:
df = gpd.read_file(r'/Users/main/Vault/Thesis/Data/BestuurlijkeGebieden_2023_voorlopig.gpkg')
placeholder = grouped_df[grouped_df["YYYYMMDD"] == 20200310]
geo_data = df.merge(placeholder, left_on="naam", right_on="tag")
fig, ax = plt.subplots(1, 1)
geo_data.plot(column='pm25', ax=ax, legend=True)

In [9]:
Locations = grouped_df["tag"].unique()
LocDict = dict()

for i in range(len(Locations)):
    LocDict[Locations[i]] = (grouped_df[grouped_df.tag == Locations[i]]["latitude"].mean(), 
    grouped_df[grouped_df.tag == Locations[i]]["longitude"].mean())
    
    
grouped_df.drop(columns=["timestamp", "latitude", "longitude"], inplace=True)

In [10]:
def get_bearing(coor1, coor2):
    dLon = (coor2[1] - coor1[1])
    y = math.sin(dLon) * math.cos(coor2[0])
    x = math.cos(coor1[0]) * math.sin(coor2[0]) - math.sin(coor1[0]) * math.cos(coor2[0]) * math.cos(dLon)
    brng = math.atan2(y, x)
    brng = np.rad2deg(brng)
    return brng

W = np.zeros((len(LocDict), len(LocDict)))
AngleMatrix = np.zeros((len(LocDict), len(LocDict)))

for i in range(len(LocDict)):
    for j in range(len(LocDict)):
        if i != j:
            theta = get_bearing(LocDict[Locations[i]], LocDict[Locations[j]])
            W[i, j] = great_circle(LocDict[Locations[i]], LocDict[Locations[j]]).km
            AngleMatrix[i, j] = theta

In [11]:
UniqueNames = grouped_df.tag.unique()

PolDict = {elem : pd.DataFrame() for elem in UniqueNames}
AngleDict = {elem : pd.DataFrame() for elem in UniqueNames}
WindDict = {elem : pd.DataFrame() for elem in UniqueNames}

for key in PolDict.keys():
    PolDict[key] = grouped_df[:][grouped_df.tag == key]
    PolDict[key].rename(columns={"pm25":key}, inplace=True)
    PolDict[key].drop(["Angle", "Wind"], axis=1, inplace=True)
    del PolDict[key]["tag"]

    AngleDict[key] = grouped_df[:][grouped_df.tag == key]
    AngleDict[key].rename(columns={"Angle":key}, inplace=True)
    AngleDict[key].drop(["pm25", "Wind"], axis=1 , inplace=True)
    del AngleDict[key]["tag"]

    WindDict[key] = grouped_df[:][grouped_df.tag == key]
    WindDict[key].rename(columns={"Wind":key}, inplace=True)
    WindDict[key].drop(["pm25", "Angle"], axis=1 , inplace=True)
    del WindDict[key]["tag"]

In [12]:
df_pol = pd.DataFrame()
df_angle = pd.DataFrame()
df_wind = pd.DataFrame()

for key in PolDict:
    df_pol = df_pol.combine_first(PolDict[key])
    df_angle = df_angle.combine_first(AngleDict[key])
    df_wind = df_wind.combine_first(WindDict[key])

In [13]:
for column in df_pol:
    median_values = (df_pol[column].median(), df_angle[column].median(), df_wind[column].median())
    df_pol[column].fillna(value=median_values[0], inplace = True)
    df_angle[column].fillna(value=median_values[1], inplace = True)
    df_wind[column].fillna(value=median_values[2], inplace = True)

In [ ]:
geo_data_array= np.zeros((len(filtered_pol)*len(df_pol.columns), 5))
for i in range(len(df_pol)):
    for i in range(5):
        geo_data_array[i, 0]

In [14]:
loc_leg = list(df_pol.columns)

In [ ]:
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6), (ax7, ax8, ax9), 
(ax10, ax11, ax12)) = plt.subplots(nrows=4, ncols=3, 
    sharey=True, figsize=(50,30))

ax1.plot(df_pol[df_pol.columns[0]])
ax1.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax1.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax1.set_xticks(df_pol.index[::2160])
ax1.set_xticklabels(df_pol.index[::2160], rotation=25)
ax1.set_title(df_pol.columns[0])

ax2.plot(df_pol[df_pol.columns[1]])
ax2.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax2.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax2.set_xticks(df_pol.index[::2160])
ax2.set_xticklabels(df_pol.index[::2160], rotation=25)
ax2.set_title(df_pol.columns[1])

ax3.plot(df_pol[df_pol.columns[2]])
ax3.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax3.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax3.set_xticks(df_pol.index[::2160])
ax3.set_xticklabels(df_pol.index[::2160], rotation=25)
ax3.set_title(df_pol.columns[2])

ax4.plot(df_pol[df_pol.columns[3]])
ax4.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax4.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax4.set_xticks(df_pol.index[::2160])
ax4.set_xticklabels(df_pol.index[::2160], rotation=25)
ax4.set_title(df_pol.columns[3])

ax5.plot(df_pol[df_pol.columns[4]])
ax5.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax5.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax5.set_xticks(df_pol.index[::2160])
ax5.set_xticklabels(df_pol.index[::2160], rotation=25)
ax5.set_title(df_pol.columns[4])

ax6.plot(df_pol[df_pol.columns[5]])
ax6.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax6.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax6.set_xticks(df_pol.index[::2160])
ax6.set_xticklabels(df_pol.index[::2160], rotation=25)
ax6.set_title(df_pol.columns[5])

ax7.plot(df_pol[df_pol.columns[6]])
ax7.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax7.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax7.set_xticks(df_pol.index[::2160])
ax7.set_xticklabels(df_pol.index[::2160], rotation=25)
ax7.set_title(df_pol.columns[6])

ax8.plot(df_pol[df_pol.columns[7]])
ax8.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax8.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax8.set_xticks(df_pol.index[::2160])
ax8.set_xticklabels(df_pol.index[::2160], rotation=25)
ax8.set_title(df_pol.columns[7])

ax9.plot(df_pol[df_pol.columns[8]])
ax9.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax9.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax9.set_xticks(df_pol.index[::2160])
ax9.set_xticklabels(df_pol.index[::2160], rotation=25)
ax9.set_title(df_pol.columns[8])

ax10.plot(df_pol[df_pol.columns[9]])
ax10.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax10.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax10.set_xticks(df_pol.index[::2160])
ax10.set_xticklabels(df_pol.index[::2160], rotation=25)
ax10.set_title(df_pol.columns[9])

ax11.plot(df_pol[df_pol.columns[10]])
ax11.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax11.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax11.set_xticks(df_pol.index[::2160])
ax11.set_xticklabels(df_pol.index[::2160], rotation=25)
ax11.set_title(df_pol.columns[10])

ax12.plot(df_pol, label=df_pol.columns)
ax12.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax12.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax12.set_xticks(df_pol.index[::2160])
ax12.set_xticklabels(df_pol.index[::2160], rotation=25)
ax12.set_title("PM2.5 Pollution Levels")

plt.savefig('Pol.png')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(50,20))
ax.plot(df_pol)
ax.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax.set_xticks(df_pol.index[::720])
ax.set_xticklabels(df_pol.index[::720], rotation=45)
ax.set_title("PM2.5 Pollution Levels")
ax.legend(loc_leg)
plt.savefig('agPol.png')
plt.show()

In [18]:
df_pol.drop(["Uithoorn", "Valsen-Zuid", "Wijk aan Zee"], axis=1, inplace=True)

KeyError: "['Uithoorn', 'Valsen-Zuid', 'Wijk aan Zee'] not found in axis"

In [15]:
for i in range(len(df_angle)):
    for j in range(len(df_angle.columns)):
        while df_angle.iloc[i, j] > 360:
            df_angle.iloc[i, j] -= 360

In [16]:
WW = np.zeros((len(df_pol), len(df_angle.columns), len(df_angle.columns)))
WWY = np.zeros((len(df_pol), len(df_pol.columns)))


filtered_pol = df_pol.copy()
for column in filtered_pol:
    filtered_pol[column] = hampel(filtered_pol[column], window_size=12, n=3, imputation=True)


for i in range(len(df_angle)):
    
    for j in range(len(df_angle.columns)):
        
        for k in range(len(df_angle.columns)):
            
            if W[j, k] != 0:
                WW[i, j, k] = np.cos(AngleMatrix[j, k] - df_angle.iloc[i, j]) * df_wind.iloc[i, j] / W[j, k]
            else:
                WW[i, j, k] = 0

    WWY[i, :] = np.matmul(WW[i, :, :], filtered_pol.iloc[i, :].to_numpy().T)


WWY_lagged = np.roll(WWY, 1, axis=0)
WWY_lagged[0, :] = 0


SWVAR = VAR(filtered_pol, exog=WWY_lagged).fit(maxlags=1, trend='c')
print(SWVAR.summary())


for key in PolDict:
    R2 = skm.r2_score(SWVAR.fittedvalues[key] + SWVAR.resid[key], SWVAR.fittedvalues[key])
    print(F'The R-Squared of {key} is: {R2*100:.2f}%')

KeyboardInterrupt: 

In [ ]:
fitted_values = SWVA

In [ ]:
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6), (ax7, ax8, ax9),
 (ax10, ax11, ax12)) = plt.subplots(nrows=4, ncols=3, 
    sharey=True, figsize=(50,30))

ax1.plot(filtered_pol[filtered_pol.columns[0]])
ax1.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax1.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax1.set_xticks(filtered_pol.index[::2160])
ax1.set_xticklabels(filtered_pol.index[::2160], rotation=25)
ax1.set_title(filtered_pol.columns[0])

ax2.plot(filtered_pol[filtered_pol.columns[1]])
ax2.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax2.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax2.set_xticks(filtered_pol.index[::2160])
ax2.set_xticklabels(filtered_pol.index[::2160], rotation=25)
ax2.set_title(filtered_pol.columns[1])

ax3.plot(filtered_pol[filtered_pol.columns[2]])
ax3.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax3.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax3.set_xticks(filtered_pol.index[::2160])
ax3.set_xticklabels(filtered_pol.index[::2160], rotation=25)
ax3.set_title(filtered_pol.columns[2])

ax4.plot(filtered_pol[filtered_pol.columns[3]])
ax4.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax4.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax4.set_xticks(filtered_pol.index[::2160])
ax4.set_xticklabels(filtered_pol.index[::2160], rotation=25)
ax4.set_title(filtered_pol.columns[3])

ax5.plot(filtered_pol[filtered_pol.columns[4]])
ax5.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax5.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax5.set_xticks(filtered_pol.index[::2160])
ax5.set_xticklabels(filtered_pol.index[::2160], rotation=25)
ax5.set_title(filtered_pol.columns[4])

ax6.plot(filtered_pol[filtered_pol.columns[5]])
ax6.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax6.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax6.set_xticks(filtered_pol.index[::2160])
ax6.set_xticklabels(filtered_pol.index[::2160], rotation=25)
ax6.set_title(filtered_pol.columns[5])

ax7.plot(filtered_pol[filtered_pol.columns[6]])
ax7.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax7.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax7.set_xticks(filtered_pol.index[::2160])
ax7.set_xticklabels(filtered_pol.index[::2160], rotation=25)
ax7.set_title(filtered_pol.columns[6])

ax8.plot(filtered_pol[filtered_pol.columns[7]])
ax8.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax8.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax8.set_xticks(filtered_pol.index[::2160])
ax8.set_xticklabels(filtered_pol.index[::2160], rotation=25)
ax8.set_title(filtered_pol.columns[7])

ax9.plot(filtered_pol[filtered_pol.columns[8]])
ax9.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax9.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax9.set_xticks(filtered_pol.index[::2160])
ax9.set_xticklabels(filtered_pol.index[::2160], rotation=25)
ax9.set_title(filtered_pol.columns[8])

ax10.plot(filtered_pol[filtered_pol.columns[9]])
ax10.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax10.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax10.set_xticks(filtered_pol.index[::2160])
ax10.set_xticklabels(filtered_pol.index[::2160], rotation=25)
ax10.set_title(filtered_pol.columns[9])

ax11.plot(filtered_pol[filtered_pol.columns[10]])
ax11.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax11.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax11.set_xticks(filtered_pol.index[::2160])
ax11.set_xticklabels(filtered_pol.index[::2160], rotation=25)
ax11.set_title(filtered_pol.columns[10])

ax12.plot(filtered_pol, label=filtered_pol.columns)
ax12.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax12.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax12.set_xticks(filtered_pol.index[::2160])
ax12.set_xticklabels(filtered_pol.index[::2160], rotation=25)
ax12.set_title("Filtered Pollution")

plt.savefig('Pol_Filtered.png')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(50,20))
ax.plot(filtered_pol)
ax.axvline(pd.to_datetime('2020-03-01 01:00:00+00:00'), color='red', linestyle='-')
ax.axvline(pd.to_datetime('2022-02-14 01:00:00+00:00'), color='red', linestyle='-')
ax.set_xticks(filtered_pol.index[::720])
ax.set_xticklabels(filtered_pol.index[::720], rotation=45)
ax.set_title("PM2.5 Pollution Levels")
ax.legend(loc_leg)
plt.savefig('agFilPol.png')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(50,20))
ax.plot(df_wind["Amsterdam"])
ax.set_xticks(df_pol.index[::720])
ax.set_xticklabels(df_pol.index[::720], rotation=45)
ax.set_title("Wind Speed")
plt.savefig('WS.png')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(50,20))
ax.plot(df_angle["Amsterdam"])
ax.set_xticks(df_pol.index[::720])
ax.set_xticklabels(df_pol.index[::720], rotation=45)
ax.set_title("Wind Direction")
plt.savefig('WD.png')
plt.show()

In [ ]:
# res = SWVAR.resid

# res_model = VAR(endog=res).fit(maxlags=1)
# # res_model.summary()


# fitted_res = res_model.fittedvalues
# first_rows = np.zeros((2, 11))
# fitted_res = np.concatenate((first_rows, fitted_res), axis=0)


# XMA = np.concatenate((WWY_lagged, fitted_res), axis=1)
# SWVARMA = VAR(filtered_pol, exog=XMA).fit(maxlags=1, trend='c')
# print(SWVARMA.summary())

# for key in PolDict:
#     R2 = skm.r2_score(SWVARMA.fittedvalues[key] + SWVARMA.resid[key], SWVARMA.fittedvalues[key])
#     print(F'The R-Squared of {key} is: {R2*100:.2f}%')

In [ ]:
zeta = SWVAR.params.to_numpy()[1:12, :]

# How to find the parameters of the environmental factors.

# alpha = np.zeros((2, 1))